In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle 

In [ ]:
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

with open('Label_Encoder.pkl','rb') as f:
    label_encoder = pickle.load(f)

with open('OneHot_Encoder.pkl','rb') as f:
    onehot_encoder = pickle.load(f)
    

In [4]:
model = load_model("Churn_ANN_model.h5")

In [6]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [7]:
input_df = pd.DataFrame([input_data])#passing dictionary as a list to make it appear as a row not dictionary

In [8]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [9]:
input_df['Gender'] = label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [12]:
geo_encoder = onehot_encoder.transform(input_df[['Geography']]).toarray()
geo_encoder = pd.DataFrame(geo_encoder,columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_encoder

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
input_df = pd.concat([input_df.drop(['Geography'],axis=1),geo_encoder],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
scaled_input = scaler.transform(input_df)
scaled_input

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [18]:
prediction = model.predict(scaled_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


In [19]:
prediction_proba = prediction[0][0]
if prediction_proba > 0.5:
    print(f"The customer is likely to churn with a probability of {prediction_proba:.2f}")
else:
    print(f"The customer is unlikely to churn with a probability of {prediction_proba:.2f}")    
    

The customer is unlikely to churn with a probability of 0.03
